<a href="https://colab.research.google.com/github/zitelli/Imersao_GeminiAI/blob/main/Gerenciador_de_Senha_%5BImers%C3%A3o_IAGemini%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
!pip install -U -q google-generativeai

In [149]:
!pip install cryptography

In [150]:
#Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai
import json
import base64

from google.colab import userdata
from cryptography.fernet import Fernet

GOOGLE_API_KEY = userdata.get('MY_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Funções responsáveis por criptografar e descriptografar o arquivo de senhas:
Se deve transfir aqui para o espaço de arquivos do colab, o arquivo de senhas (nao criptografado, formato json) denominado vault.txt.
Este será criptografado pela library Fernet para um outro arquivo denominado vault.cri

In [151]:
# Gerando uma chave de criptografia
def gerar_chave():
    chave = Fernet.generate_key()
    with open("chave.key", "wb") as chave_arquivo:
        chave_arquivo.write(chave)

# Carregando a chave de criptografia
def carregar_chave():
    return open("chave.key", "rb").read()

# Criptografando um arquivo JSON
def criptografar_json(arquivo_json, chave):
    fernet = Fernet(chave)
    with open(arquivo_json, "rb") as arquivo:
        dados = arquivo.read()
        dados_cripto = fernet.encrypt(dados)
    with open("vault.cri", "wb") as arq_criptografado:
        arq_criptografado.write(dados_cripto)

# Descriptografando o arquivo JSON
def descriptografar_json(arquivo_cripto, chave):
    fernet = Fernet(chave)
    with open(arquivo_cripto, "rb") as arquivo:
        dados_cripto = arquivo.read()
        dados_descriptografados = fernet.decrypt(dados_cripto)
    with open("vault.txt", "wb") as arquivo_descripto:
        arquivo_descripto.write(dados_descriptografados)

# Função para ler o conteúdo do arquivo JSON
def ler_arquivo_json(arquivo):
    with open(arquivo, 'r') as arq:
        conteudo_json = json.load(arq)
    return conteudo_json

In [152]:
# Gerando a chave e criptografando o arquivo JSON (gerado o arquivo chave.key)
# Este arquivo chave.key deverá ser guardado juntamente com o arquivo vault.cri, para posterior descriptografia.
gerar_chave()
chave = carregar_chave()

In [153]:
# O arquivo vault.txt que contém dados no formato JSON
arquivo_original = 'vault.txt'
# Criptografa o arquivo vault.txt gerando o arquivo vault.cri
# E uma vez este criado, pode se pagar o arquivo vault.txt, para não expor sua senha.
criptografar_json(arquivo_original, chave)

In [154]:
# Descriptografando o arquivo para gerar o arquivo vault.txt
descriptografar_json("vault.cri", chave)

In [155]:
#Exemplo de embedding
data = {"key": "Google", "description": "Senha do Google", "value": "12345678"}

embeddings = genai.embed_content(model="models/embedding-001",
                                 title=data["key"],
                                 content=data["description"]+data["value"],
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.036334958, -0.027215198, -0.04105513, -0.028980145, 0.064204685, 0.010185184, -0.011304377, -0.029812578, -0.018322708, 0.027649483, -0.0035044355, 0.017526582, -0.04635912, -0.024065144, -0.005397906, 0.017826138, -0.0073526716, -0.008521523, 0.006252347, 0.0065906844, -0.0015964945, 0.0016109896, -0.048163462, 0.0051826295, 0.030705944, -0.061658017, 0.0038036236, -0.027074473, -0.029856378, 0.035448503, -0.004826454, 0.011936607, -0.027984612, 0.0103251785, -0.008397239, -0.06407117, -0.0019585972, 0.009336097, 0.009938546, 0.0077123903, 0.0033721325, -0.050911497, -0.014298303, 0.018681288, 0.0037791235, 0.030814245, 0.0033485158, 0.017569829, 0.054024644, -0.07069078, 0.028079227, 0.019136641, 0.12856776, -0.016647529, -0.0042297975, -0.030034052, 0.031143688, 0.020906394, 0.0070004975, 0.0013619689, 0.017356168, -0.0052756667, 0.04997311, 0.023690054, -0.045258544, -0.04335225, -0.036140855, -0.004007638, 0.04057177, 0.024735417, -0.015203827, -0.055998433, 0.001

In [156]:
#Listagem de documentos que serão buscados
DOCUMENT = ler_arquivo_json('vault.txt')
documents = [DOCUMENT]
print (documents)

[{'data': [{'key': 'WhatsApp', 'description': 'Senha do WhatsApp', 'value': 'abcdef'}, {'key': 'Windows', 'description': 'Senha do Windows', 'value': 'ghijkp'}, {'key': 'Ubuntu', 'description': 'Senha do Ubuntu', 'value': 'papai_precisa'}, {'key': 'Alura', 'description': 'Senha da plataforma Alura', 'value': 'mamae_mandou'}]}]


In [157]:
df = pd.DataFrame(documents[0]['data'])
df

,key,description,value
0,WhatsApp,Senha do WhatsApp,abcdef
1,Windows,Senha do Windows,ghijkp
2,Ubuntu,Senha do Ubuntu,papai_precisa
3,Alura,Senha da plataforma Alura,mamae_mandou


In [158]:
model = "models/embedding-001"

In [164]:
def embed_fn(title, content):
  return genai.embed_content(model=model,
                                 title = title,
                                 content=content,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [165]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["key"],row["description"]+row["value"]), axis=1)
df

,key,description,value,Embeddings
0,WhatsApp,Senha do WhatsApp,abcdef,"[0.047836278, -0.0293822, -0.040392186, -0.033..."
1,Windows,Senha do Windows,ghijkp,"[0.05569444, 0.0044640545, -0.058548044, 0.000..."
2,Ubuntu,Senha do Ubuntu,papai_precisa,"[0.017314045, -0.035272848, -0.00872033, -0.03..."
3,Alura,Senha da plataforma Alura,mamae_mandou,"[0.069626525, -0.024199111, -0.023777528, -0.0..."


In [166]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["value"]

Consulta por uma senha:

In [167]:
consulta = "Alura"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

mamae_mandou
